This notebook is designed and tested using RunPod.com
Pytorch 2.0.1 template
  runpod/pytorch:2.0.1-py3.10-cuda11.8.0-devel-ubuntu22.04

In [ ]:
##############################################################################################################
# set environment variables
##############################################################################################################
import os

print("Setting environment variables...")
os.environ['GIT_REPO'] = 'https://github.com/TechTailor32/roop-unleashed.git'
os.environ['NAME'] = 'ROOP'
os.environ['WORKINGFOLDER'] = '/workspace/roop-unleashed'  # Change to a valid path

# config_cpu.yaml (CPU only) config_gpu.yaml (GPU) config_colab.yaml (Google Colab) config.yaml (Kaggle)
os.environ['CONFIGFILE'] = 'config_cuda.yaml'  # this default is config_cuda.yaml assuming you have GPU

# Output Status to Screen
print("Environment variables set:")
print("GIT_REPO:", os.environ['GIT_REPO'])
print("NAME:", os.environ['NAME'])
print("WORKINGFOLDER:", os.environ['WORKINGFOLDER'])
print("CONFIGFILE:", os.environ['CONFIGFILE'])




In [ ]:
##############################################################################################################
# install Git for using a Git repo and show status on screen
##############################################################################################################

print("Installing GitPython library...")
!pip install gitpython
print("GitPython installed successfully.")


In [ ]:
##############################################################################################################
# clone the repo (set in the Environment Variables section)
#   and output status to screen
##############################################################################################################

from git import Repo

print("Cloning the repository...")
Repo.clone_from(os.environ['GIT_REPO'], os.environ['WORKINGFOLDER'])
print(f"Repository cloned successfully into {os.environ['WORKINGFOLDER']}")


In [ ]:
##############################################################################################################
# Show current folder -> Change to the working directory if not there
##############################################################################################################

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Check if the current directory is the same as the WORKINGFOLDER
if current_directory != os.environ['WORKINGFOLDER']:
    print("Changing directory to the WORKINGFOLDER...")
    os.chdir(os.environ['WORKINGFOLDER'])
    # Confirm the change of directory
    print("New Current Working Directory:", os.getcwd())
else:
    print("Already in the WORKINGFOLDER.")


In [ ]:
##############################################################################################################
# swap the config file based on choices in Environment Variables section
##############################################################################################################

import shutil

print("Moving configuration file...")
source_path = os.path.join(os.environ['WORKINGFOLDER'], os.environ['CONFIGFILE'])
destination_path = os.path.join(os.environ['WORKINGFOLDER'], 'config.yaml')
shutil.move(source_path, destination_path)
print(f"Moved {os.environ['CONFIGFILE']} to {destination_path}")


In [ ]:
##############################################################################################################
# install extra dependencies and libraries
##############################################################################################################

import os

print("Updating system and installing required packages...")
!apt-get update
!apt-get install -y libgl1-mesa-glx ffmpeg
print("Packages installed successfully.")


In [ ]:
##############################################################################################################
# isntall dependencies based on the requirements.txt in the repo
##############################################################################################################

print("Installing required Python packages from requirements.txt...")
!pip install --no-cache-dir -r '{os.environ["WORKINGFOLDER"]}/requirements.txt'
print("Python packages installed.")


In [ ]:
##############################################################################################################
#  the repo also has Gradio support.  The following also adds an Ngrok option
##############################################################################################################

Ngrok_token = "2bgBSZ0C0BreyEthOM2fFz0T8HS_3iv1tACxTnD8JaV5EP2bZ" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)

Ngrok_domain = "" # optional, leave empty if you don't have a domain

##########################################################################

!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(7860 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system("python run.py")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')